In [ ]:
import os,cv2
from tqdm import tqdm
import numpy as np
from transformers import AutoFeatureExtractor,TFViTMAEModel

In [ ]:
features_model = AutoFeatureExtractor.from_pretrained("facebook/vit-mae-base")

In [ ]:
path = "comofod/"
data = []
labels = []
for folders in os.listdir(path):
    folder = path + folders
    print(folder)
    for items in tqdm(os.listdir(folder)):
        item = folder + '/' + items
        try:
            image = cv2.resize(cv2.imread(item),(128,128))       
            #input_ = features_model(image,return_tensors="tf")['pixel_values'].numpy()[0]
            #nimage = cv2.resize(input_,(64,64))
            #nimage = input_.T
            data.append(image)
            if folder == 'comofod/morphed':
                labels.append(1)
            else:
                labels.append(0)
        except:
            pass

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import optimizers

In [ ]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)

In [ ]:
len(data),len(labels)

In [ ]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(labels,2)

In [ ]:
from sklearn.model_selection import train_test_split

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

valAug = ImageDataGenerator()

In [ ]:
from tensorflow.keras.applications import ResNet50

model = ResNet50(weights = 'imagenet', include_top=False, input_shape=(128, 128, 3))
flat = Flatten()(model.layers[-2].output)
output = Dense(2, activation='softmax')(flat)
model = Model(model.input, output)
model.summary()

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs',write_graph=True, write_images=False)

In [ ]:
# H = model.fit(x=trainAug.flow(trainX, trainY, batch_size=8),steps_per_epoch=len(trainX) // 8, validation_data=valAug.flow(testX, testY),
# validation_steps=len(testX) // 8,epochs=25,callbacks=[tensorboard])

H = model.fit(trainX, trainY, batch_size=1, validation_data=(testX, testY),epochs=2,callbacks=[tensorboard])

In [ ]:
model.save('morph_cam.h5')

In [ ]:
acc = model.evaluate(testX,testY, verbose=0)
print("Test Loss is: ", acc[0])
print("Test accuracy is: ", acc[1])

In [ ]:
y_pred = model.predict(testX)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(testY, axis=1)

In [ ]:
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)   
    A =(((C.T)/(C.sum(axis=1))).T)    
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))    
    labels = [0,1]
    cmap=sns.light_palette("blue")    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")    
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import confusion_matrix
import seaborn as sns

plot_confusion_matrix(y_test, y_pred)

In [ ]:
last = "conv5_block3_3_conv"

In [ ]:
import tensorflow as tf

IMAGE_PATH = 'test/casia2_morphed.jpg'
LAYER_NAME = 'conv5_block3_add'

image =cv2.imread(IMAGE_PATH)
#input_ = features_model(image,return_tensors="tf")['pixel_values'].numpy()[0]
#nimage = input_.T
img1 = np.array(cv2.resize(image, (128,128)),dtype="float32") / 255.0
img2 = np.expand_dims(img1,axis=0)
#img2 = np.array(img2,dtype="float32") / 255.0
i = model.predict(img2)[0]
print(i)
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img1]))
    loss = predictions[:,0]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]  * 1e+08
gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (image.shape[1], image.shape[0]))
denom = (cam.max() - cam.min()) + 1e-16
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / denom
cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
print(image.shape, cam.shape)
output_image = cv2.addWeighted(cv2.cvtColor(image.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

cv2.imwrite('7.png', output_image)